In [1]:
import pandas as pd
import numpy as np

import sqlite3

names = ('Badges', 'PostLinks', 'Tags', 'Votes', 'Comments', 'Posts', 'Users')

for name in names:
    print(name)
    print(
        pd.read_csv(f"Exercise02/{name}.csv").head()
    )
    print('-'*10)

Badges
   Class                     Date  Id            Name  TagBased  UserId
0      3  2010-08-25T19:26:49.423   1  Autobiographer     False       2
1      3  2010-08-25T19:26:49.440   2  Autobiographer     False       3
2      3  2010-08-25T19:26:49.457   3  Autobiographer     False       4
3      3  2010-08-25T19:41:49.283   4  Autobiographer     False       6
4      3  2010-08-25T19:41:49.283   5  Autobiographer     False       7
----------
PostLinks
              CreationDate  Id  LinkTypeId  PostId  RelatedPostId
0  2010-08-26T20:33:50.797  31           1     262            261
1  2010-08-29T21:51:09.093  47           1     401            244
2  2010-08-30T01:57:19.137  48           1     132              3
3  2010-08-31T19:13:55.077  65           1     367             27
4  2010-08-31T19:13:55.077  66           1     367            246
----------
Tags
   Count  ExcerptPostId  Id    TagName  WikiPostId
0     21            NaN   1   schrader         NaN
1     44            NaN   

In [2]:
conn = sqlite3.connect('pandas_exercises')

for name in names:
    df = pd.read_csv(f"Exercise02/{name}.csv")
    df.to_sql(name, con=conn, if_exists='replace')

In [3]:
dfs = {
    name: pd.read_csv(f"Exercise02/{name}.csv")
    for name in names
}

In [4]:
query1 = """
--- Zadanie 1
SELECT PostId, COUNT(*) AS UpVotes
FROM Votes
WHERE VoteTypeId=2
GROUP BY PostId
"""

In [5]:
pd.read_sql_query(query1, con=conn)

,PostId,UpVotes
0,1,42
1,2,13
2,3,10
3,4,16
4,5,31
...,...,...
41776,63933,2
41777,63934,1
41778,63939,1
41779,63940,1


In [6]:
query2 = """
--- Zadanie 2
SELECT Title, Score, ViewCount, FavoriteCount
FROM Posts
WHERE PostTypeId=1 AND FavoriteCount >= 25 AND ViewCount>=10000
"""

#### Rozwiązanie


In [8]:
posts_df = dfs['Posts']
posts_df = posts_df[(posts_df['PostTypeId'] == 1) 
                    & (posts_df['FavoriteCount'] >= 25)
                    & (posts_df['ViewCount'] >= 10000)]
posts_df[['Title', 'Score', 'ViewCount', 'FavoriteCount']]

,Title,Score,ViewCount,FavoriteCount
220,Terminology index - a list of bike part names ...,165,129577.0,136.0
281,How to clean up at work after a commute?,101,40669.0,25.0
533,What's your most useful cycle accessory?,35,10024.0,30.0
1819,Why ride a fixed-gear bike?,171,196543.0,46.0
7643,How do I deal with a dog chasing me when I'm t...,109,50035.0,32.0


#### Sprawdzenie

In [11]:
pd.read_sql_query(query2, conn)

,Title,Score,ViewCount,FavoriteCount
0,Terminology index - a list of bike part names ...,165,129577.0,136.0
1,How to clean up at work after a commute?,101,40669.0,25.0
2,What's your most useful cycle accessory?,35,10024.0,30.0
3,Why ride a fixed-gear bike?,171,196543.0,46.0
4,How do I deal with a dog chasing me when I'm t...,109,50035.0,32.0


In [23]:
query3 = """
--- Zadanie 3
SELECT Tags.TagName, Tags.Count, Posts.OwnerUserId,
Users.Location, Users.DisplayName
FROM Tags
JOIN Posts ON Posts.Id=Tags.WikiPostId
JOIN Users ON Users.AccountId=Posts.OwnerUserId
WHERE OwnerUserId != -1
ORDER BY Count DESC
"""

In [24]:
tags_df = dfs['Tags']
posts_df = dfs['Posts']
users_df = dfs['Users']

posts_df = posts_df[posts_df['OwnerUserId'] != -1]  # filtr przed joinem optymalizuje działanie

tags_posts_df = tags_df.merge(posts_df, left_on='WikiPostId', right_on='Id')
tags_posts_users_df = tags_posts_df.merge(users_df, left_on='OwnerUserId', right_on='AccountId')

(tags_posts_users_df
 .sort_values('Count', ascending=False)
 [['TagName', 'Count', 'OwnerUserId', 'Location', 'DisplayName']]
)

,TagName,Count,OwnerUserId,Location,DisplayName
0,freewheel,167,3021.0,"Moscow, Russia",jfs
2,preventative-maintenance,62,7044.0,"Manchester, United Kingdom",Codebeef
1,7-speed,21,7044.0,"Manchester, United Kingdom",Codebeef
6,lbs,15,3649.0,NaN,Tim
7,reliability,5,3927.0,"Liverpool, United Kingdom",Will Boyce
5,sociable,2,7044.0,"Manchester, United Kingdom",Codebeef
3,ordinary,1,7044.0,"Manchester, United Kingdom",Codebeef
4,dicycle,1,7044.0,"Manchester, United Kingdom",Codebeef


In [26]:
pd.read_sql_query(query3, conn)

,TagName,Count,OwnerUserId,Location,DisplayName
0,freewheel,167,3021.0,"Moscow, Russia",jfs
1,preventative-maintenance,62,7044.0,"Manchester, United Kingdom",Codebeef
2,7-speed,21,7044.0,"Manchester, United Kingdom",Codebeef
3,lbs,15,3649.0,None,Tim
4,reliability,5,3927.0,"Liverpool, United Kingdom",Will Boyce
5,sociable,2,7044.0,"Manchester, United Kingdom",Codebeef
6,ordinary,1,7044.0,"Manchester, United Kingdom",Codebeef
7,dicycle,1,7044.0,"Manchester, United Kingdom",Codebeef


In [27]:
query4 = """
--- Zadanie 4
SELECT Posts.Title, RelatedTab.NumLinks FROM
(SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
FROM PostLinks GROUP BY RelatedPostId) AS RelatedTab
JOIN Posts ON RelatedTab.PostId=Posts.Id
WHERE Posts.PostTypeId=1
ORDER BY NumLinks DESC
"""

In [42]:
postlinks_df = dfs['PostLinks']
posts_df = dfs['Posts']

related_tab_df = (postlinks_df
 .groupby('RelatedPostId')[['PostId']]
 .count()
 .reset_index()
 .rename(columns={
     'PostId': 'NumLinks',
     'RelatedPostId': 'PostId'
    })
)

posts_df = posts_df[posts_df['PostTypeId'] == 1]

(related_tab_df
 .merge(posts_df, left_on='PostId', right_on='Id')
 [['Title', 'NumLinks']]
 .sort_values('NumLinks', ascending=False)
)

,Title,NumLinks
42,Terminology index - a list of bike part names ...,156
2064,Why shouldn't I care what model/make/year my b...,154
1995,How can I tell what year my bike was made?,100
2077,What is the maximum or minimum tire width I ca...,42
627,Front suspension or full suspension mountain bike,37
...,...,...
406,I am about to buy a new bicycle - what should ...,1
849,Disassembly and Shipping of bike,1
1340,Rotating two chains on one cassette,1
850,I was in a collision the other day. What shoul...,1


In [43]:
pd.read_sql_query(query4, conn)

,Title,NumLinks
0,Terminology index - a list of bike part names ...,156
1,Why shouldn't I care what model/make/year my b...,154
2,How can I tell what year my bike was made?,100
3,What is the maximum or minimum tire width I ca...,42
4,Front suspension or full suspension mountain bike,37
...,...,...
2202,Total brake failure - trying to understand wha...,1
2203,Sram Red AXS Crank Bolt Cap missing - how to r...,1
2204,Chain Waxing - Strainer / Colander / Other?,1
2205,Can I use 26 1 3/8 tire to replace a 27 1 1/4?,1


In [44]:
query5 = """
--- Zadanie 5
SELECT UpVotesTab.*, Posts.Title FROM
(
SELECT PostId, COUNT(*) AS UpVotes FROM Votes WHERE VoteTypeId=2 GROUP BY PostId
) AS UpVotesTab
JOIN Posts ON UpVotesTab.PostId=Posts.Id
WHERE Posts.PostTypeId=1
ORDER BY UpVotesTab.UpVotes DESC
"""

In [83]:
votes_df = dfs['Votes']
posts_df = dfs['Posts']

upvotes_df = (votes_df
 [votes_df['VoteTypeId'] == 2]
 .groupby('PostId')[['Id']]
 .count()
 .reset_index()
 .rename(columns={'Id': 'UpVotes'})
)

posts_df = posts_df[posts_df['PostTypeId'] == 1]

(upvotes_df
 .merge(posts_df, left_on='PostId', right_on='Id')
 [list(upvotes_df.columns) + ['Title']]
 .sort_values('UpVotes', ascending=False)
)

,PostId,UpVotes,Title
393,1980,174,Why ride a fixed-gear bike?
70,244,167,Terminology index - a list of bike part names ...
1643,8233,110,How do I deal with a dog chasing me when I'm t...
83,308,103,How to clean up at work after a commute?
454,2294,94,"Safety data: Which is safer, head/tail lights ..."
...,...,...,...
3355,16947,1,Are New York pedestrians allowed to use bike l...
5154,29049,1,Retrofit lockout for a suspension fork?
5155,29060,1,Can a 7 speed Derailleur work on a 8 speed cas...
5158,29072,1,3 flats in 650 miles (1030 km) on hybrid bike....


In [84]:
pd.read_sql_query(query5, conn)

,PostId,UpVotes,Title
0,1980,174,Why ride a fixed-gear bike?
1,244,167,Terminology index - a list of bike part names ...
2,8233,110,How do I deal with a dog chasing me when I'm t...
3,308,103,How to clean up at work after a commute?
4,2294,94,"Safety data: Which is safer, head/tail lights ..."
...,...,...,...
11977,63912,1,DTSwiss rear wheel hub part name
11978,63918,1,Identification of non-standard frame feature a...
11979,63923,1,Proper use of shimano chain measuring tool
11980,63927,1,I'm a fairly new mountain bike rider. Doing a ...


In [82]:
query6 = """
--- Zadanie 6
SELECT UpVotesTab.PostId, UpVotesTab.UpVotes, IFNULL(DownVotesTab.DownVotes, 0) AS DownVotes
FROM
(
SELECT PostId, COUNT(*) AS UpVotes FROM Votes
WHERE VoteTypeId=2 GROUP BY PostId
) AS UpVotesTab
LEFT JOIN
(SELECT PostId, COUNT(*) AS DownVotes FROM Votes
WHERE VoteTypeId=3 GROUP BY PostId
) AS DownVotesTab
ON UpVotesTab.PostId=DownVotesTab.PostId
"""

In [90]:
# Zauważamy, że upvotes_df mamy z poprzedniego zadania

downvotes_df = (votes_df
 [votes_df['VoteTypeId'] == 3]
 .groupby('PostId')[['Id']]
 .count()
 .reset_index()
 .rename(columns={'Id': 'DownVotes'})
)

(upvotes_df
 .merge(downvotes_df, how='left', on='PostId')
 .fillna(0)
)

,PostId,UpVotes,DownVotes
0,1,42,0.0
1,2,13,0.0
2,3,10,0.0
3,4,16,0.0
4,5,31,0.0
...,...,...,...
41776,63933,2,0.0
41777,63934,1,0.0
41778,63939,1,0.0
41779,63940,1,0.0


In [91]:
pd.read_sql_query(query6, conn)

,PostId,UpVotes,DownVotes
0,1,42,0
1,2,13,0
2,3,10,0
3,4,16,0
4,5,31,0
...,...,...,...
41776,63933,2,0
41777,63934,1,0
41778,63939,1,0
41779,63940,1,0


In [92]:
query7 = """
--- Zadanie 7
SELECT PostId, UpVotes-DownVotes AS Votes FROM (
SELECT UpVotesTab.PostId, UpVotesTab.UpVotes, IFNULL(DownVotesTab.DownVotes, 0) AS DownVotes
FROM
(
SELECT PostId, COUNT(*) AS UpVotes FROM Votes
WHERE VoteTypeId=2 GROUP BY PostId
) AS UpVotesTab
LEFT JOIN
(
SELECT PostId, COUNT(*) AS DownVotes
FROM Votes WHERE VoteTypeId=3 GROUP BY PostId
) AS DownVotesTab
ON UpVotesTab.PostId=DownVotesTab.PostId
UNION
SELECT DownVotesTab.PostId, IFNULL(UpVotesTab.UpVotes, 0) AS UpVotes, DownVotesTab.DownVotes
FROM
(
SELECT PostId, COUNT(*) AS DownVotes FROM Votes
WHERE VoteTypeId=3 GROUP BY PostId
) AS DownVotesTab
LEFT JOIN
(
SELECT PostId, COUNT(*) AS UpVotes FROM Votes
WHERE VoteTypeId=2 GROUP BY PostId
) AS UpVotesTab
ON DownVotesTab.PostId=UpVotesTab.PostId
)
"""

In [98]:
### W tym zadaniu wyliczamy ocenę każdego postu (upvotes - downvotes),
### robimy to w taki sposób aby uwzględnić sytuacje
### w których post ma same upvotes lub same downvotes

downvotes_df = (votes_df
 [votes_df['VoteTypeId'] == 3]
 .groupby('PostId')[['Id']]
 .count()
 .reset_index()
 .rename(columns={'Id': 'DownVotes'})
)

upvotes_downvotes_df = pd.concat([
    (upvotes_df
     .merge(downvotes_df, how='left', on='PostId')
     .fillna(0)
    ),
    (downvotes_df
     .merge(upvotes_df, how='left', on='PostId')
     .fillna(0))
])

upvotes_downvotes_df['Votes'] = upvotes_downvotes_df['UpVotes'] - upvotes_downvotes_df['DownVotes']
upvotes_downvotes_df.groupby('PostId')[['Votes']].sum().reset_index()

,PostId,Votes
0,1,42.0
1,2,13.0
2,3,10.0
3,4,16.0
4,5,31.0
...,...,...
45704,63933,2.0
45705,63934,1.0
45706,63939,1.0
45707,63940,1.0


In [100]:
pd.read_sql_query(query7, conn)

,PostId,Votes
0,1,42
1,2,13
2,3,10
3,4,16
4,5,31
...,...,...
45704,63933,2
45705,63934,1
45706,63939,1
45707,63940,1
